<a href="https://colab.research.google.com/github/JayThibs/ai-safety-scrape/blob/main/scrape_ai_alignment_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Contents from AI Alignment Resources

This notebook is used to scrape the contents of AI Alignment resources.

# Installations

In [1]:
!python --version

Python 3.7.12


In [2]:
%env PYTHONPATH=

env: PYTHONPATH=


In [3]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-py37_4.11.0-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
Unpacking payload ...
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py37h27cfd23_1003
    - ca-certificates==2021.10.26=h06a4308_2
    - certifi==2021.10.8=py37h06a4308_2
    - cffi==1.15.0=py37hd667e15_1
    - charset-normalizer==2.0.4=pyhd3eb1b0_0
    - conda-content-trust==0.1.1=pyhd3eb1b0_0
    - conda-package-handling==1.7.3=py37h27cfd23_1
    - conda==4.11.0=py37h06a4308_0
    - cryptography==36.0.0=py37h9ce1e76_0
    - idna==3.3=pyhd3eb1b0_0
    - ld_impl_linux-64==2.35.1=h7274673_9
    - libffi==3.3=he6710b0_2
    - libgcc-ng==9.3.0=h5101ec6_17
    - libgomp==9.3.0=h5101ec6_17
    - libstdcxx-ng==9.3.0=hd4cf53a_17
    - ncurses==6.3=h7f8727e_2
    - openssl==1.1.1m=h7f8727e_0
    - pip==21.2.2=py37h06a4308_0
    - pycosat==0.6.3=py37h27cfd23_0
    - pycparser==2.21=pyhd3eb1b0_0
    - pyopenssl==21.0.0=py

--2022-03-02 19:50:45--  https://repo.continuum.io/miniconda/Miniconda3-py37_4.11.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-py37_4.11.0-Linux-x86_64.sh [following]
--2022-03-02 19:50:45--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.11.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103730670 (99M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.11.0-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 57.0M 2s
    50K .......... .......... .......... 

In [4]:
%%bash

conda install --channel defaults conda python=3.7 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - conda
    - python=3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.2.1   |       h06a4308_0         122 KB
    ------------------------------------------------------------
                                           Total:         122 KB

The following packages will be UPDATED:

  ca-certificates                     2021.10.26-h06a4308_2 --> 2022.2.1-h06a4308_0



ca-certificates-2022 | 122 KB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local


The following packages will be downloaded:

    package                    |            build
    ---

In [5]:
import sys

_ = (sys.path.append("/usr/local/lib/python3.7/site-packages"))

In [6]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/usr/local/lib/python3.7/site-packages']

In [7]:
!conda install -c conda-forge pypandoc --yes

Solving environment: / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pypandoc


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2021.10.8          |   py37h89c1867_1         145 KB  conda-forge
    conda-4.11.0               |   py37h89c1867_0        16.9 MB  conda-forge
    pandoc-2.17.1.1            |       ha770c72_0        11.9 MB  conda-forge
    pypandoc-1.7.2             |     pyhd8ed1ab_0          26 KB  conda-forge
    python_abi-3.7             |          2_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        29.1 MB

The following NEW packages will be INSTALLED:

  pandoc             conda-forge/linux-64::pandoc

In [8]:
!conda install -c anaconda pip --yes

Solving environment: \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pip


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.10.14 |                0         128 KB  anaconda
    certifi-2020.6.20          |           py37_0         159 KB  anaconda
    pip-20.2.4                 |           py37_0         2.0 MB  anaconda
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2021.10.~ --> anaconda::ca-certificates-2020.10.14-0
  certifi            conda-forge::certifi-2021.10.8-py37h8~ --> anaconda::certifi-2020.6.20-py37_0
  conda              conda-forge::conda-4.11.0-py37h89c186~ --> pkgs/main::conda-4.11

In [9]:
!pip install --upgrade pandoc -q

     |████████████████████████████████| 117 kB 8.1 MB/s 
     |████████████████████████████████| 49 kB 4.0 MB/s 


In [10]:
!pip install tika arxiv

     |████████████████████████████████| 81 kB 3.9 MB/s 
  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32893 sha256=9ba09f64ce29f9010edc323c9029c8c094d0bd152017dbceb8649a3ba668c06d
  Stored in directory: /root/.cache/pip/wheels/ec/2b/38/58ff05467a742e32f67f5d0de048fa046e764e2fbb25ac93f3
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=88a8549faff180ab6a2cafbadb3fa06d8aaec805cf3d8b9485dcc1790eac3b70
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36cebfdddfe059f50e019aa
Successfully built tika sgmllib3k


# Imports

In [11]:
import os
import pandas as pd
from tika import parser
from google.colab import drive
from pathlib import Path
from urllib import request
from bs4 import BeautifulSoup
import arxiv
import pypandoc
import requests
import tarfile
import pickle
from urllib.error import HTTPError

# Setting up Environment

In [12]:
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [30]:
CODE_DIR = Path('.') / 'code-projects/gpt-ai-safety'
RAW_DIR = Path('data/raw')
INTERIM_DIR = Path('data/interim')
PROCESSED_DIR = Path('data/processed')
TARS_DIR = RAW_DIR / 'tars'
LATEX_DIR = RAW_DIR / 'latex_files'
PDFS_DIR = RAW_DIR / 'pdfs'
PKLS_DIR = INTERIM_DIR / 'pkls'
EXTRACTED_TARS_DIR = INTERIM_DIR / 'extracted_tars'
MERGE_TEX_DIR = INTERIM_DIR / 'merge_latex_files'
PROCESSED_TXTS_DIR = PROCESSED_DIR / 'txts'

In [15]:
!mkdir -p {CODE_DIR}

In [34]:
%cd {CODE_DIR}

[Errno 2] No such file or directory: 'code-projects/gpt-ai-safety'
/content/drive/MyDrive/code-projects/gpt-ai-safety


In [33]:
!mkdir -p {RAW_DIR} {INTERIM_DIR} {PROCESSED_DIR} {TARS_DIR} {LATEX_DIR} {PDFS_DIR}
!mkdir -p {PKLS_DIR} {EXTRACTED_TARS_DIR} {MERGE_TEX_DIR} {PROCESSED_TXTS_DIR}

In [35]:
if not os.path.exists('ai-alignment-papers.csv'):
    !git clone https://github.com/JayThibs/ai-safety-scrape
    !mv ai-safety-scrape/* .

# Load and Explore Data

In [36]:
df = pd.read_csv('ai-alignment-papers.csv')
df.head()

,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,Abstract Note,Date,Date Added,Date Modified,Access Date,Pages,Num Pages,Issue,Volume,Number Of Volumes,Journal Abbreviation,Short Title,Series,Series Number,Series Text,Series Title,Publisher,Place,Language,Rights,Type,Archive,Archive Location,Library Catalog,Call Number,Extra,Notes,File Attachments,Link Attachments,Manual Tags,...,Cast Member,Commenter,Composer,Cosponsor,Counsel,Interviewer,Producer,Recipient,Reviewed Author,Scriptwriter,Words By,Guest,Number,Edition,Running Time,Scale,Medium,Artwork Size,Filing Date,Application Number,Assignee,Issuing Authority,Country,Meeting Name,Conference Name,Court,References,Reporter,Legal Status,Priority Numbers,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body
0,XBZAPQFK,blogPost,2020.0,"Kokotajlo, Daniel",Three kinds of competitiveness,AI Impacts,NaN,NaN,NaN,https://aiimpacts.org/three-kinds-of-competiti...,"By Daniel Kokotajlo In this post, I distinguis...",2020-03-30,2022-01-30 01:53:10,2022-01-30 01:53:10,2021-11-20 18:55:39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en-US,NaN,NaN,NaN,NaN,NaN,NaN,ZSCC: NoCitationData[s0] ACC: N/A Section: Blog,NaN,/Users/jacquesthibodeau/Zotero/storage/PU9A2KS...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HX9UZ5JP,journalArticle,2020.0,"Cihon, Peter; Maas, Matthijs M.; Kemp, Luke",Fragmentation and the Future: Investigating Ar...,Global Policy,NaN,1758-5899,10.1111/1758-5899.12890,https://onlinelibrary.wiley.com/doi/abs/10.111...,The international governance of artificial int...,2020,2022-01-30 04:47:43,2022-01-30 04:47:43,2021-11-13 15:58:24,545-556,NaN,5,11.0,NaN,NaN,Fragmentation and the Future,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,Wiley Online Library,NaN,ZSCC: 0000010 _eprint: https://onlinelibrary....,NaN,/Users/jacquesthibodeau/Zotero/storage/2TZBI3F...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BQCZM53S,blogPost,2021.0,"Clarke, Sam; Martin, Samuel Dylan",Distinguishing AI takeover scenarios,AI Alignment Forum,NaN,NaN,NaN,https://www.alignmentforum.org/posts/qYzqDtoQa...,Epistemic status: lots of this involves interp...,2021-09-08,2022-01-30 04:47:42,2022-01-30 04:47:42,2021-11-18 23:45:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSCC: NoCitationData[s0] ACC: N/A,NaN,/Users/jacquesthibodeau/Zotero/storage/ENAMQXC...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JVMJ4RMM,journalArticle,2020.0,"Stray, Jonathan",Aligning AI Optimization to Community Well-Being,International Journal of Community Well-Being,NaN,"2524-5295, 2524-5309",10.1007/s42413-020-00086-3,http://link.springer.com/10.1007/s42413-020-00...,NaN,2020-12,2022-01-30 04:47:36,2022-01-30 04:47:36,2021-11-13 22:47:54,443-463,NaN,4,3.0,NaN,Int. Journal of Com. WB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,DOI.org (Crossref),NaN,ZSCC: 0000010,NaN,/Users/jacquesthibodeau/Zotero/storage/V3BEV7X...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,W8F6VI9I,thesis,2020.0,"Shah, Rohin Monish",Extracting and Using Preference Information fr...,NaN,NaN,NaN,NaN,https://www.proquest.com/openview/da8bf63ef343...,Typically when learning about what people want...,2020-12-17,2022-01-30 04:47:35,2022-01-30 04:47:35,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"University of California, Berkeley","Berkeley, CA",en,NaN,NaN,NaN,NaN,Zotero,NaN,ZSCC: 0000000,NaN,/

In [37]:
df['Item Type'].unique()

array(['blogPost', 'journalArticle', 'thesis', 'conferencePaper',
       'manuscript', 'report', 'bookSection', 'magazineArticle', 'book'],
      dtype=object)

In [38]:
item_nums = []
for item in df['Item Type'].unique():
    item_nums.append([item, len(df[df['Item Type'] == item])])

item_nums.sort(key=lambda x:x[1])
item_nums

[['magazineArticle', 2],
 ['thesis', 3],
 ['book', 13],
 ['bookSection', 52],
 ['report', 87],
 ['manuscript', 154],
 ['journalArticle', 170],
 ['conferencePaper', 262],
 ['blogPost', 421]]

In [39]:
df[df['Item Type'] == 'journalArticle'].head()

,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,Abstract Note,Date,Date Added,Date Modified,Access Date,Pages,Num Pages,Issue,Volume,Number Of Volumes,Journal Abbreviation,Short Title,Series,Series Number,Series Text,Series Title,Publisher,Place,Language,Rights,Type,Archive,Archive Location,Library Catalog,Call Number,Extra,Notes,File Attachments,Link Attachments,Manual Tags,...,Cast Member,Commenter,Composer,Cosponsor,Counsel,Interviewer,Producer,Recipient,Reviewed Author,Scriptwriter,Words By,Guest,Number,Edition,Running Time,Scale,Medium,Artwork Size,Filing Date,Application Number,Assignee,Issuing Authority,Country,Meeting Name,Conference Name,Court,References,Reporter,Legal Status,Priority Numbers,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body
1,HX9UZ5JP,journalArticle,2020.0,"Cihon, Peter; Maas, Matthijs M.; Kemp, Luke",Fragmentation and the Future: Investigating Ar...,Global Policy,NaN,1758-5899,10.1111/1758-5899.12890,https://onlinelibrary.wiley.com/doi/abs/10.111...,The international governance of artificial int...,2020,2022-01-30 04:47:43,2022-01-30 04:47:43,2021-11-13 15:58:24,545-556,NaN,5,11.0,NaN,NaN,Fragmentation and the Future,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,Wiley Online Library,NaN,ZSCC: 0000010 _eprint: https://onlinelibrary....,NaN,/Users/jacquesthibodeau/Zotero/storage/2TZBI3F...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JVMJ4RMM,journalArticle,2020.0,"Stray, Jonathan",Aligning AI Optimization to Community Well-Being,International Journal of Community Well-Being,NaN,"2524-5295, 2524-5309",10.1007/s42413-020-00086-3,http://link.springer.com/10.1007/s42413-020-00...,NaN,2020-12,2022-01-30 04:47:36,2022-01-30 04:47:36,2021-11-13 22:47:54,443-463,NaN,4,3.0,NaN,Int. Journal of Com. WB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,DOI.org (Crossref),NaN,ZSCC: 0000010,NaN,/Users/jacquesthibodeau/Zotero/storage/V3BEV7X...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,TK5F29IU,journalArticle,2021.0,"Hayden, Benjamin; Niv, Yael",The case against economic values in the orbito...,Behavioral Neuroscience,NaN,NaN,10.1037/bne0000448,https://osf.io/7hgup,Much of traditional neuroeconomics proceeds fr...,2021,2022-01-30 04:48:47,2022-01-30 04:48:47,2021-11-08 23:41:47,192-201,NaN,2,135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DOI.org (Crossref),NaN,ZSCC: 0000026 DOI: 10.31234/osf.io/7hgup,NaN,NaN,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NHWZIKZ2,journalArticle,2020.0,"Fernandes, Pedro; Santos, Francisco C.; Lopes,...",Norms for Beneficial A.I.: A Computational Ana...,AI Communications,NaN,"18758452, 09217126",10.3233/AIC-201502,http://arxiv.org/abs/1907.03843,The rise of artificial intelligence (A.I.) bas...,2020-12-18,2022-01-30 04:48:46,2022-01-30 04:48:46,2021-11-13 22:40:37,155-171,NaN,3-6,33.0,NaN,AIC,Norms for Beneficial A.I.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,arXiv.org,NaN,ZSCC: 0000004 arXiv: 1907.03843,NaN,/Users/jacquesthibodeau/Zotero/storage/JAVXSVN...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,HDWGJGAP,journalArticle,2021.0,"Mingard, Chris; Valle-Pérez, Guillermo; Skalse...","Is SGD a Bayesian sampler? Well, almost",Journal of Machine Learning Research,NaN,NaN,NaN,http://arxiv.org/abs/2006.15191,Overparameterised deep neural networks (DNNs) ...,2021-02,2022-01-30 04:48:46,2022-01-30 04:48:46,2021-11-13 22:56:31,NaN,NaN,NaN,22.

# Extract text from AI Alignment Resources

In [40]:
df_arxiv = df[df['Url'].str.contains('arxiv') == True]
arxiv_paper_list = df_arxiv['Url'].values
arxiv_paper_list[0:20]

array(['http://arxiv.org/abs/2002.11328',
       'http://arxiv.org/abs/2010.11645',
       'http://arxiv.org/abs/2002.11708',
       'http://arxiv.org/abs/2012.10800',
       'http://arxiv.org/abs/2011.08512',
       'http://arxiv.org/abs/2002.11089',
       'http://arxiv.org/abs/2012.05876',
       'http://arxiv.org/abs/2010.14603',
       'http://arxiv.org/abs/2005.01643',
       'http://arxiv.org/abs/1907.03843',
       'http://arxiv.org/abs/2009.12612',
       'http://arxiv.org/abs/2007.08124',
       'http://arxiv.org/abs/2006.15191',
       'http://arxiv.org/abs/1909.13371',
       'http://arxiv.org/abs/2008.08076',
       'http://arxiv.org/abs/2009.00802',
       'http://arxiv.org/abs/2011.08541',
       'http://arxiv.org/abs/2009.08092',
       'http://arxiv.org/abs/2003.07305',
       'http://arxiv.org/abs/2004.09044'], dtype=object)

In [41]:
arxiv_paper_tars = []
incorrect_links_ids = []
for paper_link in arxiv_paper_list:
    try:
        paper_id = paper_link.split('/')[-1]
        paper = next(arxiv.Search(id_list=[paper_id]).results())
        tar_filename = paper.entry_id.split('/')[-1] + '.tar.gz'
        arxiv_paper_tars.append(tar_filename)
    except:
        incorrect_links_ids.append([paper_link, paper_id])
        pass

In [42]:
with open(PKLS_DIR / 'arxiv_paper_tars_list.pkl', 'wb') as f:
    pickle.dump(arxiv_paper_tars, f)

In [ ]:
paper_dl_failures = []
for paper_link, paper_tar in zip(arxiv_paper_list, arxiv_paper_tars):
    if os.path.exists(str(TARS_DIR / paper_tar)):
        print('tar file already exists')
        continue
    paper_id = paper_link.split('/')[-1]
    paper = next(arxiv.Search(id_list=[paper_id]).results())
    filename = paper.entry_id.split('/')[-1] + '.tar.gz'
    if not os.path.exists(str(TARS_DIR / filename)):
        try:
            paper.download_source(dirpath='tars', filename=filename)
        except HTTPError:
            print('Could not download paper: ' + paper.entry_id)
            paper_dl_failures.append(paper.entry_id)
            pass
    else:
        print('Already downloaded: ' + paper.entry_id)

tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already exists
tar file already

HTTPError: Page request resulted in HTTP 400: incorrect id format for 0512204v2 (http://export.arxiv.org/api/query?search_query=&id_list=0512204v2&sortBy=relevance&sortOrder=descending&start=0&max_results=100)

In [ ]:
print(paper_dl_failures)

In [ ]:
# # Grabbing files from ar5iv in case source (LaTeX doesn't work)

# def grab_text_from_webpage(url):
#     with request.urlopen(url) as response:
#         html = response.read()
#     soup = BeautifulSoup(html, "html.parser")
#     return soup.get_text(separator=" ")

# df_arxiv['Url'].str.replace('arxiv', 'ar5iv')
# arxiv_paper = grab_text_from_webpage('http://ar5iv.org/abs/2002.11328')
# arxiv_paper_list = arxiv_paper.split('\n\n')
# list_text = BeautifulSoup(arxiv_paper, "html.parser").get_text(separator=" ").split('\n')
# ''.join(list_text)

## Extract LaTeX papers from arXiv

In [ ]:
paper = next(arxiv.Search(id_list=["2007.08124v1"]).results())

In [ ]:
paper.entry_id

'http://arxiv.org/abs/2007.08124v1'

In [ ]:
def download_arxiv_paper_tar(paper_id, tar_filename):
    # paper = next(arxiv.Search(id_list=[paper_id]).results())
    # print(f'Paper Entry ID: {paper.entry_id}')
    if not os.path.exists(str(TARS_DIR / tar_filename)):
        print('Downloading paper source...')
        paper.download_source(dirpath=TARS_DIR, filename=tar_filename)

def extract_tex(tar_filename):
    tar = tarfile.open(str(TARS_DIR / tar_filename))
    tex_filename = None
    for member in tar.getmembers():
        if ".tex" in member.name:
            tex_filename = member.name
            tar.extract(member, LATEX_DIR)
    tar.close()
    return tex_filename

In [ ]:
pickled_arxiv_list = str(RAW_DIR / "arxiv_ids_and_names.pkl")

if not os.path.exists(pickled_arxiv_list):
    arxiv_ids_and_names = []
else:
    pickle.load(pickled_arxiv_list)

In [ ]:
paper.download_source(dirpath=TARS_DIR, filename=tar_filename)

HTTPError: ignored

In [ ]:
arxiv_ids_and_names = []
pdf_ids = []
for paper_id in arxiv_paper_list:
    paper_id_v1 = paper_id.split('/')[-1] + "v1"
    # print(f'paper_id: {paper_id_v1}')
    tar_filename = paper_id_v1 + ".tar.gz"
    # print(f'tar_filename: {tar_filename}')
    download_arxiv_paper_tar(paper_id_v1, tar_filename)
    # print("downloaded")
    if not os.path.exists(str(LATEX_DIR / tar_filename)):
        tex_filename = extract_tex(tar_filename)
    if tex_filename:
        arxiv_ids_and_names.append([paper_id, tex_filename])
    else: 
        #Is pdf only, do the pdf-scraper tiki later
        pdf_ids.append(paper_id)

Paper Entry ID: http://arxiv.org/abs/2009.12612v1
Couldn't download paper.
Paper Entry ID: http://arxiv.org/abs/2007.08124v1
Couldn't download paper.


In [ ]:
with open('arxiv_ids_and_names.pkl', 'wb') as b:
    pickle.dump(arxiv_ids_and_names, b)

## Convert LaTeX or EPUB with Pandoc

In [ ]:
class ExitCodeError(Exception): pass

In [ ]:
def sh(x):
    if os.system(x):
        raise ExitCodeError()

In [ ]:
def texToTxt(filename, format='latex'):
    try:
        pypandoc.convert_file(filename, to='plain', format=format, outputfile=filename[:-3] + 'txt')
    except:
        if not os.path.exists('fallback_pile'):
            os.mkdir('fallback_pile')
        sh(f'mv {filename} fallback_pile/')
        pass
        # import traceback
        # traceback.print_exc()

In [ ]:
texToMarkdown("data/raw/latex_files/introduction.tex")

In [ ]:
isinstance(b, str)

True

In [ ]:
import chardet

def fwrite(fname, content):
    with open(fname, "w") as fh:
        fh.write(content)


def any_to_utf8(b):
    if not isinstance(b, str):
        try:
            return b.decode("utf-8")
        except:
            # try to figure out encoding if not utf-8
            pass
            guess = chardet.detect(b)["encoding"]

            if not guess or guess == "UTF-8":
                return

            try:
                return b.decode(guess)
            except (UnicodeDecodeError, LookupError):
                # still cant figure out encoding, give up
                return

In [ ]:
!mkdir -p data/raw/latex_files/out done fallback_needed errored

In [ ]:
class join:
    def __init__(self, sep):
        self.sep = sep

    def __rrshift__(self, other):
        if other is None:
            return
        try:
            return self.sep.join(other)
        except:
            return None

In [ ]:
filename = "algorithm"
tex = str(LATEX_DIR / filename) + ".tex"
out_name = str(LATEX_DIR / "out" / filename) + ".txt"

try:
    with open(tex, 'rb') as fh:
        b = fh.read()
        print(b)
        cont = any_to_utf8(b)
        print(cont)
    fwrite(tex, cont)
except:
    pass

try:
    sh(f'timeout 10s pandoc -s {tex} -o {out_name}  --wrap=none')
    # print(os.path.exists(f'out/{out_name}.md'))
except ExitCodeError:
    import traceback
    traceback.print_exc()
    # fallback:
    try:
        # move to fallback pile so we can handle it later
        if '_extract' in tex.split('/')[:-1] >> join('/'):
            loc = tex.split('/')[:-1] >> join('/')
        else:
            loc = tex
        sh(f'mv {loc} fallback_needed/')

    except ExitCodeError:
        import traceback
        traceback.print_exc()

b"\\section{Scalable and Efficient SDP-relaxation Solver}\n\\label{sec:scale-SDP}\nOur goal is to develop a custom solver for large-scale neural network verification with the following desiderata:\n(1) compute \\emph{anytime} upper bounds valid after each iteration, %\n(2) rely on elementary computations with efficient implementations that can exploit hardware like GPUs and TPUs,\nand (3) have per-iteration memory and computational cost that scales linearly in the number of neurons.\n\nIn order to satisfy these desiderata, we employ first order methods to solve the Langrange dual problem~\\eqref{eq:lag-standard}. We derive a reformulation of the Lagrange dual with only non-negativity constraints on the decision variables (Section~\\ref{sec:bc}). We then show how to efficiently and conveniently compute subgradients of the objective function in Section~\\ref{sec:lanczos} and derive our final solver in Algorithm~\\ref{alg:verify}.\n\n\\subsection{Reformulation to a problem with only non-n

In [ ]:
tex.split('/')[:-1] >> join('/')

'data/raw/latex_files'

## Extracting PDFs with Tika

In [ ]:
def tikaTextExtractor(file_path):
    """Extracts text from a PDF using tika."""
    print("Extracting text from file: " + file_path)
    parsed_tika = parser.from_file(file_path)
    return parsed_tika["content"]

txt = tikaTextExtractor(str(RAW_DIR / 'pdfs' / 'Superintelligence.pdf'))

In [ ]:
print(txt[0:1000])

In [ ]:
with open(PROCESSED_TXTS_DIR / 'Superintelligence.txt', 'w') as f:
    f.write(txt)

# Search all useful arxiv papers

In [ ]:
search = arxiv.Search(
    query="ai safety",
    max_results=10,
    sort_by = arxiv.SortCriterion.Relevance
)

In [ ]:
for result in search.results():
  print('Title: ', result.title, '\nDate: ',result.published , '\nId: ', result.entry_id, 
        '\nSummary: ',result.summary ,'\nURL: ', result.pdf_url, '\n\n')

Title:  Evolutionary Computation and AI Safety: Research Problems Impeding Routine and Safe Real-world Application of Evolution 
Date:  2019-06-24 19:26:10+00:00 
Id:  http://arxiv.org/abs/1906.10189v2 
Summary:  Recent developments in artificial intelligence and machine learning have
spurred interest in the growing field of AI safety, which studies how to
prevent human-harming accidents when deploying AI systems. This paper thus
explores the intersection of AI safety with evolutionary computation, to show
how safety issues arise in evolutionary computation and how understanding from
evolutionary computational and biological evolution can inform the broader
study of AI safety. 
URL:  http://arxiv.org/pdf/1906.10189v2 


Title:  AI Failures: A Review of Underlying Issues 
Date:  2020-07-18 15:31:29+00:00 
Id:  http://arxiv.org/abs/2008.04073v1 
Summary:  Instances of Artificial Intelligence (AI) systems failing to deliver
consistent, satisfactory performance are legion. We investigate w

# SOTAWHAT

In [ ]:
!git clone https://github.com/chiphuyen/sotawhat
%cd sotawhat
!pip install .
%cd ..

Cloning into 'sotawhat'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 92 (delta 0), reused 1 (delta 0), pack-reused 89
Unpacking objects: 100% (92/92), done.
/content/drive/MyDrive/code-projects/gpt-ai-safety/sotawhat
Processing /content/drive/MyDrive/code-projects/gpt-ai-safety/sotawhat
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 2.7 MB 5.1 MB/s 
  Created wheel for sotawhat: filename=sotawhat-0.0.1-py3-none-any.whl size=4623 sha256=5aef8419ceec258d0ead1cf1725627fcb3cf7f5d4dfae58b5c359b5929819a17
  St

In [ ]:
!sotawhat ai safety 10

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Impossibility Results in AI: A Survey (Mario Brcic - 1 September, 2021)
As such, these results serve as guidelines, reminders, and warnings to AI safety, AI policy, and governance researchers. We concluded that deductive impossibilities deny 100%-guarantees for security
Link: sp;[<a href=
Towards a framework for evaluating the safety, acceptability and efficacy of AI systems for health: an initial synthesis (Jessica Morley - 14 April, 2021)
There are numerous reasons for this, but a main contributory factor is the lack of internationally accepted, or formalised, regulatory standards to assess AI safety and impact and effectiveness. We do this by conducting a systematic search across Scopus, PubMed and Google Scholar to identify all the relevant literature published between January 1970 and November 2020 related to the evaluation of: output performance; efficacy; and real-world use 